# Coordinate Calculator and Label Compiler

Inputs:
- location in pixels of beginning and end of axes
- range of axes
- units of axis range
- user's desired spacing for x coordinates
- NO legend input! how to specify different colors/markers that separates the curves (for later stages)
    - could eventually have mask look for the legend (distinctive enough) to learn what types of distinguishing features exist between the lines on the graph
    - I will ignore this part for now
- detectron2 output:
    - list of dicts, one dict for each image
    - each dict has field 'panoptic_seg' which refers to a (named?) tuple.
    - the first field in the tuple is a tensor with the ID of each pixel
    - the second field is a list of dicts with one dict per ID.
    - entries in this dict are 'id' which is the ID that matches the first field tensor, and 'category_id'
    
Outputs:
- dictionary with the following fields:
    - coordinates: a dictionary, with fields equal to the curve labels (one for each label), and entries that are a list of tuples, each with (x, and y coordinates in same units as input image)
    - units: (a tuple of x and y units)
    - beginning and end of curve
    - NO labels (for later stages): a list of labels for each curve, taken from legend if possible, or automatic (data1, data2, etc)


Options for coordinates:
1. based on the actual axis, choose a set of x coordinates for which we want to know the y coordinates (for each curve)
2. based on pixels, grab points that occur every "few" pixels along the x axis, then convert to coordinates

- could do the first option by finding out how many pixels are between each of the given x points, and using that number of pixels paired with the second option.
- do we want user to input the spacing between points, or should we use a default value that seems reasonable based on the range of the x axis?
- should the different curves have uniform x axis values (so y values are directly comparable), or should we have the sets of data points for each curve start at the 'beginning' of the curve and end at the 'end'


Outlier cases:
- nested plots
- weird legend formatting

In [2]:
# uniform x axis but also save the very first and last endpoints of the curve 

In [ ]:
# establish scaling from pixel to real units
def get_x_scale():
    return x_scale


def get_y_scale():
    return y_scale

In [ ]:
# convert pixel location to coordinates
def pixel_to_coords():
    return coords

In [ ]:
# each curve is one ID. get all pixel locations for one ID
def get_pixels_for_id(ID):
    return pixels

In [ ]:
# convert ID to the label from the legend input
def get_label_for_id():
    return label

In [ ]:
# create coordinate dictionary to add to the output dict
def create_coordinate_dict():
    # initialize dict
    # for each ID:
        #get_pixels_for_id(ID)
    return coordinate_dict

In [1]:
# create the output dictionary and input the units of axis and labels directly from the input info,
# as well as the coordinate dictionary
def create_output_dict():
    return